In [1]:
import torch
from sklearn.model_selection import train_test_split
from torch import nn
import os 
import csv
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from torch.utils.data import DataLoader, TensorDataset
from scipy.ndimage import gaussian_filter1d
from scipy.signal import find_peaks
from sklearn.metrics import confusion_matrix


# Device Setup

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Optionally, you can print more details about the GPU if available
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"Number of GPUs available: {torch.cuda.device_count()}")

Using device: cuda
GPU Name: Tesla T4
Number of GPUs available: 1


## Single Model Training


In [6]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
from src.utils import  preprocess_data_for_classifier, train_model
from src.models import STNClassifier, MLPClassifier,  CNNClassifier

max_shift = 3.0 # in eV
max_broadening_sigma = 0.5


train_loader, test_loader, input_features, output_features = preprocess_data_for_classifier(
    spectra_dir= '../data/synthetic_data_hard_final/synthetic_spectra',
    labels_dir='../data/synthetic_data_hard_final/synthetic_labels',
    max_shift=max_shift,
    max_broadening_sigma=max_broadening_sigma,
    valence_range=0,  #valence region is already removed in saved data
    energy_resolution=0.1, 
    test_size=0.2,
    random_state=42,
    batch_size=64)
# 4. Create an instance of the model and send it to target device

#model_0 =  STNClassifier().to(device)

model_0 = STNClassifier().to(device)
# Load the saved model
model_path = Path("../models/STNClassifier_shift_3.0eV_model.pth")


model_0.load_state_dict(torch.load(model_path))



loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model_0.parameters(), lr=1e-6)

# Calculate accuracy (a classification metric)
def accuracy_fn(y_true, y_pred):
    # Check if all 39 classes match for each sample
    correct = (y_true == y_pred).all(axis=1).sum().item()  # Use .all(axis=1) to check all classes
    acc = (correct / len(y_true)) * 100  # Calculate accuracy as a percentage
    return acc

# Create a list to store loss values
training_loss_values = []
testing_loss_values = []

scheduler = ReduceLROnPlateau(
    optimizer,
    mode='min',      # 'min' means it looks for the metric to decrease
    factor=0.1,      # Factor by which the learning rate will be reduced (new_lr = lr * factor)
    patience=5,     # Number of epochs with no improvement after which learning rate will be reduced
    verbose=True     # Prints a message when the learning rate is updated
)

# Train the model
training_information = train_model(
    model=model_0,
    train_loader=train_loader,
    test_loader=test_loader,
    loss_fn=loss_fn,
    optimizer=optimizer,
    device=device,
    epochs=100,
    print_freq=1, # Print every 100 epochs
    scheduler= scheduler
)

training_loss_values = training_information['train_loss']
testing_loss_values = training_information['test_loss']
trained_model = training_information['model']

Epoch:    0 | Train Loss: 0.03484, Acc: 67.51% | Test Loss: 0.02615, Acc: 75.37%
Epoch:    1 | Train Loss: 0.03484, Acc: 67.54% | Test Loss: 0.02649, Acc: 75.31%
Epoch:    2 | Train Loss: 0.03477, Acc: 67.64% | Test Loss: 0.02612, Acc: 75.60%
Epoch:    3 | Train Loss: 0.03468, Acc: 67.77% | Test Loss: 0.02615, Acc: 75.28%
Epoch:    4 | Train Loss: 0.03458, Acc: 67.88% | Test Loss: 0.02606, Acc: 75.66%
Epoch:    5 | Train Loss: 0.03459, Acc: 67.89% | Test Loss: 0.02588, Acc: 75.89%
Epoch:    6 | Train Loss: 0.03472, Acc: 67.76% | Test Loss: 0.02629, Acc: 75.25%
Epoch:    7 | Train Loss: 0.03462, Acc: 67.51% | Test Loss: 0.02597, Acc: 75.66%
Epoch:    8 | Train Loss: 0.03458, Acc: 67.76% | Test Loss: 0.02614, Acc: 75.61%
Epoch:    9 | Train Loss: 0.03448, Acc: 67.78% | Test Loss: 0.02619, Acc: 75.47%


KeyboardInterrupt: 